In [2]:
import pandas as pd

# Bonus 1

In [3]:
df = pd.read_csv('../data/raw/full_data.csv')
df2 = pd.read_csv('../data/processed/df_processed.csv')

,Country,Job Title,Rural
0,Austria,Not job found,Rural
1,Austria,Automatic Data Processing Planner,Non-rural
2,Austria,Not job found,Non-rural
3,Austria,Data Coordinator,Rural
4,Austria,Database Developer,Non-rural
...,...,...,...
9644,Slovakia,Data Warehouse Developer,Non-rural
9645,Slovakia,Database Manager,Non-rural
9646,Slovakia,Not job found,Non-rural
9647,Slovakia,Data Officer,Non-rural


In [151]:
def bonus1(df):
    df.rename(columns={'dem_education_level': 'Education Level',
                   'question_bbi_2016wave4_basicincome_argumentsfor': 'Pro Arguments',
                   'question_bbi_2016wave4_basicincome_argumentsagainst': 'Cons Arguments',
                  'question_bbi_2016wave4_basicincome_vote': 'Position',
                  'question_bbi_2016wave4_basicincome_effect': 'Effect',
                  'question_bbi_2016wave4_basicincome_awareness': 'Awareness'}, inplace=True)
    
    def split(x):
        return x.split('|')
    
    df['Pro Arguments'] = df.apply(lambda x: split(x['Pro Arguments']), axis=1)
    
    df['Cons Arguments'] = df.apply(lambda x: split(x['Cons Arguments']), axis=1)
    
    def ind_range(x):
        if x == ['None of the above']:
            return 0
        else:
            return len(x)
        
    df['Pro Arguments'] = df.apply(lambda x: ind_range(x['Pro Arguments']), axis=1)
    
    df['Cons Arguments'] = df.apply(lambda x: ind_range(x['Cons Arguments']), axis=1)
    
    def position(x):
        if 'vote for' in x:
            return 'In Favor'
        elif 'vote against' in x:
            return 'Against'
        else:
            return 'Impartial'
        
    df['Position'] = df.apply(lambda x: position(x['Position']), axis=1)
    
    table = df.groupby('Position')['Pro Arguments', 'Cons Arguments'].sum()
    
    table = table.rename(columns={'Pro Arguments':'Number of Pro Arguments', 'Cons Arguments':'Number of Cons Arguments'}).drop(index='Impartial')
    
    return table

In [152]:
bonus1(df)

<ipython-input-151-86f249329e3d>:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bonus1 = df.groupby('Position')['Pro Arguments', 'Cons Arguments'].sum()


,Number of Pro Arguments,Number of Cons Arguments
Position,,
Against,2195,6558
In Favor,11753,10928


# Bonus 2

In [29]:
def bonus2(df_raw, df_processed):
    
    df_raw.rename(columns={'dem_education_level': 'Education Level', 'normalized_job_code': 'Job Title'}, inplace=True)
    df_raw['Job Title'] = df_processed['Job Title']
    df_raw['Education Level'] = df_raw['Education Level'].replace('no', 'no education')

    ed_list = list(df_raw['Education Level'].fillna('None').unique())
    ed_list.remove('None')
    new_order = [1, 2, 3, 0]
    ed_list = [ed_list[i] for i in new_order]

    filtered_dfs = [df_raw[df_raw['Education Level'] == level] for level in ed_list]

    dfs_skills = [i['Job Title'].value_counts().head(11) for i in filtered_dfs]

    final_df = [i.drop(index='Not job found') for i in dfs_skills]

    table = pd.DataFrame(
        {'Education Level': ed_list, 'Top 10 Skills': [list(final_df[i].index) for i in range(len(final_df))]},
        index=None)

    table['Top 10 Skills'] = table.apply(lambda x: ', '.join(x['Top 10 Skills']), axis=1)

    return table

In [30]:
l = bonus2(df, df2)

In [31]:
l

,Education Level,Top 10 Skills
0,high,"Data Administrator, Data Officer, Data Warehou..."
1,medium,"Data Storage Specialist, Geographic Informatio..."
2,low,"Data Processing Systems Analyst, Data Communic..."
3,no education,"Database Administration Associate, Data Wareho..."
